# **ΠΡΟΒΛΗΜΑ ΤΑΞΙΝΟΜΗΣΗΣ ΑΝΤΙΚΕΙΜΕΝΩΝ 3D ΕΚΤΥΠΩΤΗ**

Ενσωματωμένος αισθητήρας επιτήρησης ενός laser, που χρησιμοποιείται στη διαδικασία 3Δ εκτύπωσης μεταλλικών αντικειμένων, μετράει σε διακριτές -7970- χρονικές στιγμές την διάσταση του αντικειμένου. Για το πρόβλημα διατίθενται 58 σήματα / δείγματα για τη διαδικασία εκπαίδευσης ενός μοντέλου που θα προβλέπει αν ένα νέο σήμα αντιστοιχεί σε φυσιολογικό, ελλαττωματικό τύπου 1 ή ελλαττωματικό τύπου 2 αντικείμενο.

Από αυτά τα στοιχεία εξάγονται σημαντικά σημασιολογικά χαρακτηριστικά τα οποία θα κρίνουν τον τρόπο προ-επεξεργασίας των δεδομένων και τη τεχνική εκπαίδευσης του μοντέλου.

Στο μέρος της προ-επεξεργασίας διαπιστώθηκαν δύο κύριοι τρόποι προσέγγισης. Σε αυτή την υλοποίηση αξιοποιείται η πρώτη επιλογή που αναφέρεται παρακάτω.


*   Κάθε διακριτό χρονικό δείγμα ενός σήματος μπορεί να θεωρηθεί σαν ξεχωριστό χαρακτηριστικό, όντας μοναδικό πάνω στον χώρο. Κατά αυτόν τον τρόπο, όσα είναι και τα χρονικά δείγματα ενός σήματος θα είναι και τα χαρακτηριστικά του.
*   Σύμφωνα με την παρατήρηση πως τα δεδομένα αποτελούν χρονικά σήματα, υπάρχει η δυνατότητα μετάβασης στον χώρο της συχνότητας με Fast-Fourier-Transform και η χρήση χαρακτηριστικών όπως το φάσμα συχνότητας, η μέση συχνότητα, ενέργεια του σήματος. Αυτή η μέθοδος θυμίζει παράλληλα και την τεχνική μείωσης διαστάσεων του προβλήματος

Στη συνέχεια γίνεται κανονικοποίηση των δεδομένων, έτσι ώστε χαρακτηριστικά μεγαλύτερης τάξης μεγέθους να μην ασκούν μεγαλύτερη επιρροή στην διαδικασία εκπαίδευσης του μοντέλου. Αυτό το ζήτημα εμφανίζεται με την προϋπόθεσή ότι χρησιμοποιούνται μετρικές απόστασης για τον προσδιορισμό των δεδομένων στον χώρο που μελετάται. Η τεχνική εκπαίδευσης που πρόκειται να εφαρμοστεί ενσωματώνει στον μαθηματικό φορμαλισμό της τέτοιες μετρικές, οπότε και το παραπάνω βήμα είναι απαραίτητο.

Επίσης αφού το πρόβλημα αφορά το ίδιο ακριβώς αντικείμενο το οποίο μπορεί να διαφέρει χωρικά, σε πολύ συγκεκριμένα σημεία του λόγω ελαττωμάτων, σημαίνει πως αρκετά από τα χαρακτηριστικά του θα παρουσιάζουν ανεπαίσθητη διακύμανση, οπότε και η συνεισφορά τους στη διακριτοποίηση των κλάσεων θα είναι -τουλάχιστον- περιττή. Οπότε μια καλή επιλογή για ταχύτερη εκπαίδευση και βελτιστοποίηση των παραμέτρων του μοντέλου, θα γίνει μείωση διαστάσεων με χρήση PCA.


In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

# Load data - we stay in the time domain
X_train = pd.read_csv('Training_Signals.csv', header=None).T # .T because every sample is a column
y_train = pd.read_csv('Training_Signals_State.csv', header=None).values.ravel() # Flatten with ravel
X_test = pd.read_csv('Test_Signals.csv', header=None).T

# Normalize features - zero mean / unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply PCA
pca = PCA(n_components=0.9)  # Retain 90% of variance
X_train = pca.fit_transform(X_train) # Learn the components and transform the training set
X_test = pca.transform(X_test) # With already known pca components transform the test set

Τα σήματα παραμένουν στο πεδίο του χρόνου οπότε:

Τα χαρακτηριστικά τους θα είναι σημαντικά πιο πολλά από τον αριθμό των σημάτων εκπαίδευσης, σε συνδυασμό με το ότι,
Το πλήθος των σημάτων εκπαίδευσης είναι ιδιαίτερα περιορισμένο

Αυτά τα μέρη του προβλήματος, καθιστούν την χρήση SVM ιδανική λόγω της σταθερότητάς του σε τέτοιες συνθήκες, αφού βασίζεται στα κρίσιμα σημεία κάθε κλάσης, για γραμμικά και μη-γραμμικά διαχωρίσιμα προβλήματα.

Δοκιμάζονται διαφορετικές παράμετροι και γίνεται αυτοματοποιημένος έλεγχος του κάθε μοντέλου με τη GridSearchCV. Είναι αναμενόμενο πως λόγω των περιορισμένων δεδομένων ο γραμμικός kernel θα δίνει καλύτερη δυνατότητα γενίκευσης σε αυτό το μοντέλο. Όσον αφορά την υπερπαράμετρο C γίνεται εποπτικός έλεγχος μεγάλου εύρους τιμών για να φανεί για το συγκεκριμένο πρόβλημα ποια είναι η κατάλληλη τάξη μεγέθους. Τυπικά στη συνέχεια θα γινόταν πειραματισμός με τιμές σε εκείνη τη τάξη μεγέθους, αλλά επειδή το μοντέλο φαίνεται να αποδίδει άριστα για όλα τα C, παραλείπεται σαν βήμα.  

Το validation έγινε με τη χρήση k fold cross validation λόγω του μικρού dataset, με μετρική την ακρίβεια επειδή φαίνεται πως οι κλάσεις απαρτίζουν το dataset με ίδιο περίπου ποσοστό .


In [2]:
# Model training with SVM and hyperparameter optimization
svm = SVC()
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf']
}

# Split training data for validation - either this or cross validation
#X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2)

# 3 / 5 fold to choose best model -> small training set - also need sufficient k fold test samples
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring="accuracy") # assesing models with accuracy
grid_search.fit(X_train, y_train) # if we don't perform k fold use X_train_split, y_train_split
best_model = grid_search.best_estimator_

# Print chosen parameters for the best model
print(best_model.get_params())

# Evaluate on separate validation set (if we used one)
''' val_predictions = best_model.predict(X_val)
print("Validation Classification Report:")
print(classification_report(y_val, val_predictions)) '''

# Retrieve cross-validation results - (if we used it)
cv_results = grid_search.cv_results_
for mean, std, params in zip(cv_results['mean_test_score'], cv_results['std_test_score'], cv_results['params']):
    print(f"Mean Accuracy: {mean:.4f} (+/- {std:.4f}) for params: {params}")

# Best model's validation accuracy
best_score = grid_search.best_score_
print(f"Best Validation Accuracy: {best_score:.4f}")

{'C': 0.01, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 0.01, 'kernel': 'linear'}
Mean Accuracy: 0.3818 (+/- 0.0594) for params: {'C': 0.01, 'kernel': 'rbf'}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 0.1, 'kernel': 'linear'}
Mean Accuracy: 0.3818 (+/- 0.0594) for params: {'C': 0.1, 'kernel': 'rbf'}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 1, 'kernel': 'linear'}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 1, 'kernel': 'rbf'}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 10, 'kernel': 'linear'}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 10, 'kernel': 'rbf'}
Mean Accuracy: 1.0000 (+/- 0.0000) for params: {'C': 100, 'kernel': 'linear'}
Mean Accuracy: 1.0000 (+/- 0.0

Τέλος παράγονται τα εκτιμώμωνα labels του προβλήματος και έχουμε και μια συνάρτηση που για κάθε επόμενο test signal που μπορεί να δωθεί, να γίνεται απευθείας η εκτίμηση της κλάσης του.

In [3]:
# Predict on test set
test_predictions = best_model.predict(X_test)

# Save predictions
test_predictions_df = pd.DataFrame(test_predictions, columns=['Prediction'])
test_predictions_df.to_csv('test_predictions_task1_58352.csv', index=False, header=False)

# Classify a single or multiple signals using the trained model.
def classify_signal(signal):
    # Normalize the signal(s)
    signal = scaler.transform([signal])
    # Apply PCA
    signal = pca.transform(signal)
    # Predict the class(es)
    return best_model.predict(signal)[0]